<a href="https://colab.research.google.com/github/Mathurshab2210/Ml_with_Mathur/blob/main/Fakenews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import re# to search the words in doc
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [4]:
news=pd.read_csv('/content/drive/MyDrive/CNN/train.csv')

In [5]:
news.describe()

,id,label
count,20800.000000,20800.000000
mean,10399.500000,0.500625
std,6004.587135,0.500012
min,0.000000,0.000000
25%,5199.750000,0.000000
50%,10399.500000,1.000000
75%,15599.250000,1.000000
max,20799.000000,1.000000


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [6]:
news.shape

(20800, 5)

In [7]:
news.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [8]:
 #missing value analysis
 news.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [9]:
#  replacing null values
news=news.fillna('')

In [10]:
 #mergin author and text
 news['content']=news['author']+' '+news['text']
 print(news['content'])


0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn Ever get the feeling your life...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss Videos 15 Civilians Killed In ...
4        Howard Portnoy Print \nAn Iranian woman has be...
                               ...                        
20795    Jerome Hudson Rapper T. I. unloaded on black c...
20796    Benjamin Hoffman When the Green Bay Packers lo...
20797    Michael J. de la Merced and Rachel Abrams The ...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799    David Swanson   David Swanson is an author, ac...
Name: content, Length: 20800, dtype: object


In [11]:
#seprating data and label
x=news.drop('label',axis=1)
y=news['label']

In [12]:
x,y

(          id                                              title  \
 0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
 1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
 2          2                  Why the Truth Might Get You Fired   
 3          3  15 Civilians Killed In Single US Airstrike Hav...   
 4          4  Iranian woman jailed for fictional unpublished...   
 ...      ...                                                ...   
 20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
 20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
 20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
 20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
 20799  20799                          What Keeps the F-35 Alive   
 
                                           author  \
 0                                  Darrell Lucus   
 1                                Daniel J. Flynn   
 2                     

Stemming:
means to find root word


In [13]:
port_stem=PorterStemmer()

In [14]:
def stemming(content):
  stem_content=re.sub('[^a-zA-Z]',' ',content)#to take alphabets and eexcluding everything else
  stem_content=stem_content.lower()
  stem_content=stem_content.split()
  stem_content=[port_stem.stem(word) for word in stem_content if not word in  stopwords.words('english')]
  stem_content=' '.join(stem_content)
  return stem_content

In [15]:
news['content']=news['content'].apply(stemming)

In [16]:
news['content']

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn ever get feel life circl rounda...
2        consortiumnew com truth might get fire octob t...
3        jessica purkiss video civilian kill singl us a...
4        howard portnoy print iranian woman sentenc six...
                               ...                        
20795    jerom hudson rapper unload black celebr met do...
20796    benjamin hoffman green bay packer lost washing...
20797    michael j de la merc rachel abram maci today g...
20798    alex ansari nato russia hold parallel exercis ...
20799    david swanson david swanson author activist jo...
Name: content, Length: 20800, dtype: object

In [17]:
#seprating data and label
x=news['content'].values
y=news['label'].values

In [18]:
print(x,y)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet darrel lucu octob subscrib jason chaffetz stump american fork utah imag courtesi michael jolley avail creativ common licens apolog keith olbermann doubt worst person world week fbi director jame comey accord hous democrat aid look like also know second worst person well turn comey sent infam letter announc fbi look email may relat hillari clinton email server rank democrat relev committe hear comey found via tweet one republican committe chairmen know comey notifi republican chairmen democrat rank member hous intellig judiciari oversight committe agenc review email recent discov order see contain classifi inform long letter went oversight committe chairman jason chaffetz set polit world ablaz tweet fbi dir inform fbi learn exist email appear pertin investig case reopen jason chaffetz jasoninthehous octob cours know case comey actual say review email light unrel case know anthoni weiner sext teenag appar littl thing f

In [19]:
y

array([1, 0, 1, ..., 0, 1, 1])

In [20]:
y.shape

(20800,)

In [21]:
#text to feature vector
vectorizer=TfidfVectorizer ()
vectorizer.fit(x)
X=vectorizer.transform(x)



In [22]:
X

<20800x111254 sparse matrix of type '<class 'numpy.float64'>'
	with 5108263 stored elements in Compressed Sparse Row format>

In [24]:
#train test split
X_train,X_test,y_train,y_test=train_test_split(X,y, test_size=0.2, stratify=y, random_state=42)

In [25]:
 model=LogisticRegression()

In [27]:
model.fit(X_train,y_train)

LogisticRegression()

In [28]:
X_train_pred=model.predict(X_train)
train_data_acc=accuracy_score(X_train_pred,y_train)

In [29]:
print(train_data_acc)

0.9745793269230769


In [31]:
X_test_pred=model.predict(X_test)
test_data_acc=accuracy_score(X_test_pred,y_test)

In [32]:
print('test acc:',test_data_acc)

test acc: 0.9487980769230769


In [33]:
#making a predictive sys

In [36]:
X_new=X_test[0]
pred=model.predict(X_new)

In [37]:
pred

array([0])

In [38]:
if pred[0] == 0:
  print("real news")
else:
  print('fake news')

real news


In [39]:
y_test[0]

0